# 作業工程計画

0. ライブラリを読み込む
1. 過去データを読み込む
2. 過去データを整形する
3. 過去データを学習してモデルを作る 
4. 設問データを読み込む
5. 設問データをモデルに読み込ませ、予測値を出す
6. 予測値を設問データに追加する
7. 6のデータに、分野別最終ペースの最大値、最小値、平均値を追加する
8. データを出力し、納品する

## ライブラリ

In [133]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
import re 

"""
機械学習ライブラリの準備
"""

from sklearn.model_selection import cross_val_score
# from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.tree import DecisionTreeRegressor #決定木
import lightgbm as lgb #lightGBM

from sklearn import tree
import graphviz

### 村上さん pandas_tool

In [3]:
"""
村上さんtoolbox
"""
# pandas 基礎集計クラス
import numpy as np
import pandas as pd
import copy
import seaborn as sns
from itertools import combinations

#http://qiita.com/tanemaki/items/2ed05e258ef4c9e6caac

# Jupyterで表示するためには、最初に以下を実行すること
%matplotlib inline

# Static Classで設計する
class pandas_tool:
    
    # All in one チェック（Jupyterのみ）
    def all_basic_summary_jupyter(df):
        print("■ 型の確認")
        display(pandas_tool.type(df))
        print("■ 数値型の確認")
        display(pandas_tool.summary(df)[0])
        print("■ カテゴリ型の確認")
        cate_var_data = list(df.select_dtypes(include=['object']).columns)
        ret = pandas_tool.freq(df,cate_var_data)
        for d in ret:
            display(pd.DataFrame(d))
            print("---------------")
        print("■ 欠損の確認")
        display(pandas_tool.check_missing(df))
    
    # 相関関係可視化（Jupyterのみ）
    def all_value_relation_visualize(df):
        #sns.set_context("poster", 1.2, {"lines.linewidth": 3})
        sns.pairplot(df,size=5)
    
    # カテゴリ変数でのヒートマップ（Jupyterのみ）
    def make_heatmap(df,x,y,value):
        target_df = df.pivot_table(index=x,values=value,columns=y)
        sns.heatmap(target_df, annot=True, fmt='1.1f', cmap='Blues')
    
    # 散布図（Jupyterのみ）
    def make_scatter_chart(df,x,y):
        #sns.jointplot(x=x, y=y, data=df, kind="hex")
        sns.jointplot(x=x, y=y, data=df)
    
    # 組み合わせでヒートマップを作成（Jupyterのみ）
    def all_make_heatmap(df,var_list,value):
        col_num = 2
        var_list_set = list(combinations(var_list,2))
        
        fig, axes = plt.subplots(int(len(var_list_set)/col_num)+1, col_num, figsize=(18,3+6.5*int(len(var_list_set)/col_num)))
        
        for i,target in enumerate(var_list_set):
            target_df = df.pivot_table(index=target[0],values=value,columns=target[1])
            sns.heatmap(target_df, annot=True, fmt='1.1f', cmap='Blues', ax=axes[int(i/col_num), i%col_num])
            
        plt.tight_layout()
    
    # 数値集計
    def summary(df,view=False):
        ret=df.describe()
        mis_ret=df.isnull().sum()
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print("・統計量")
            print(ret)
            print("・欠損値")
            print(mis_ret)
            pd.set_option("display.max_columns",param)
        return ret,mis_ret
    
    # 型チェック
    def type(df,view=False):
        ret = df.dtypes
        if view:
            param=pd.get_option("display.max_rows")
            pd.set_option("display.max_rows",1000)
            print(ret)
            pd.set_option("display.max_rows",param)
        return ret
    
    # 欠損チェック
    def check_missing(df,view=False):
        not_null_df=df.notnull()
        ret=pd.DataFrame()
        for name in not_null_df.columns:
            tmp_df=not_null_df[name].value_counts()
            tmp_df.name=name
            ret = pd.concat([ret,tmp_df],axis=1)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 欠損値のオブザベーションを抽出
    def get_miss_data(df,column,view=False):
        ret=df[df[column].isnull()]
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        return ret
    
    # 欠損値を中央値で補完
    def fill_miss_med(df,var_name):
        var=df[var_name].median()
        df[var_name].fillna(var,inplace=True)
        return df
    
    # 欠損値を0で補完
    def fill_miss_zero(df,var_name):
        df[var_name].fillna(0,inplace=True)
        return df
    
    # 特定の値を欠損とみなす
    def apply_miss_value(df,var_name,value):
        df[var_name]=df[var_name].replace(value,np.nan)
        return df
    
    # 重複チェック
    def check_dup(df,columns,view=False):
        ret=pd.DataFrame()
        for name in columns:
            dup_cnt=df[name].duplicated().sum()
            tmp_df = pd.DataFrame({'var_name':[name],'dup_cnt':[dup_cnt]})
            ret = pd.concat([ret,tmp_df],axis=0,ignore_index= True)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 組み合わせ重複チェック
    def check_dup_comb(df,columns,view=False):
        ret = df[columns].duplicated().sum()
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # ユニークデータ取得
    def get_uniq_data(df,uniq_key,sort_key,keep='first'):
        ret = df.sort_values(by=sort_key)
        ret.drop_duplicates(subset=uniq_key, keep=keep, inplace=True)
        return ret
    
    # カテゴリ集計
    def freq(df,columns,view=False):
        ret=list()
        for name in columns:
            tmp_df=df[name].value_counts()
            tmp_df.name=name
            #ret = pd.concat([ret,tmp_df],axis=1)
            ret.append(tmp_df)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            for r in ret:
                print(r)
                #display(r)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 複雑な集計
    def tabulate(df,row,col=None,var='',func=np.sum,view=False):
        if var == '':
            tmp_df=df.reset_index(drop=False,inplace=False)
            ret=pd.pivot_table(data=tmp_df, values='index', index=row, columns=col, aggfunc='count', dropna=False, fill_value=0 ,margins = False)
            tmp_df=None
        else:
            ret=pd.pivot_table(data=df, values=var, index=row, columns=col, aggfunc=func, dropna=False, fill_value=0 ,margins = False)
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # マージ
    def merge(df1,df2,key,how,view=True):
        if view:
            print("df1のキー重複")
            pandas_tool.check_dup_comb(df1,key,True)
            print("df2のキー重複")
            pandas_tool.check_dup_comb(df2,key,True)
            
            print("df1のオブザベーション:{0}".format(len(df1)))
            print("df2のオブザベーション:{0}".format(len(df2)))
        
        ret=pd.merge(df1,df2,how=how,on=key)
        
        if view:
            print("mergeのオブザベーション:{0}".format(len(ret)))
        
        return ret
    
    # Rank
    def rank(df,var,num,suffix='_rank',check=False):
        labels=[i for i in range(0,num)]
        df[var+suffix]=pd.qcut(df[var], num, labels=labels)
        
        # check data
        if check:
            ret=pd.DataFrame()
            max_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func=np.max,view=False)
            max_df.name='max'
            min_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func=np.min,view=False)
            min_df.name='min'
            cnt_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func='count',view=False)
            cnt_df.name='count'
            ret=pd.concat([ret,min_df,max_df,cnt_df],axis=1)
            return df,ret
            
        return df
    
    # Rank適用(min基準)
    def apply_rank(df,rank_df):
        tmp_df=copy.deepcopy(rank_df)
        tmp_df.reset_index(drop=False,inplace=True)
        target_name=tmp_df.columns[3]
        tmp_df.columns=["rank","min","max","cnt"]
        
        def judge_thld(row):
            ret_var = -1
            cond_list = ["if 0 : ret_var = 0"]
            
            for i in range(1,len(tmp_df)):
                cond_list.append("elif row < " +str(tmp_df.ix[i,'min'])+ " : ret_var = " + str(tmp_df.ix[i-1,'rank']))
            
            cond_list.append("else: ret_var = " + str(tmp_df.ix[len(tmp_df)-1,'rank']))
            cond_str="\r\n".join(cond_list)
            # ローカル辞書をexecと共有する
            local_dict=locals()
            exec(cond_str,local_dict)
            return local_dict["ret_var"]
        
        df[target_name+"_rank"]=df[target_name].apply(judge_thld)
        return df
    
    # Min%以下はMin%点に、Max%以上はMax%点にクリップする
    def clip_min_max(df,col_list,apply_df=None,max_pct=0.99,min_pct=0.01):
        p_min = df[col_list].quantile(min_pct)
        p_max = df[col_list].quantile(max_pct)
        
        df[col] = df[col_list].clip(p_min,p_max,axis=1)
        
        # もしも適用先のデータがあるならば（例えば検証データ）対応
        if apply_df is not None:
            apply_df[col] = apply_df[col_list].clip(p_min,p_max,axis=1)
            return df,apply_df
        else:
            return df
    
    
    # 文字列→数値変換
    def conv_float(df,column,percent_flg=False):
        
        def conv_f(row):
            if row[column] == "" or row[column] is np.nan:
                return np.nan
            else:
                return float(row[column])
        
        df[column]=df[column].str.replace("\\","").str.replace(",","").str.replace("%","").str.strip()
        df[column]=df.apply(conv_f,axis=1)
        
        if percent_flg:
            df[column]=df[column]/100
        
        return df

## 過去データの前処理

In [118]:
"""
path_name = ファイルパス
sn_list = エクセルシート名のリスト
"""
path_name = "/Users/s.ogura/Documents/CRLEA/data/rawdata/①設問情報（分野、文字数等の情報、企画ペース）_0522更新_欠損値ハイライト2_test.xlsx"
path_file =  r'{}'.format(path_name)
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names
#配列ループ
df = [] #空のリスト
for i in (0,1): #21冬

    # Excelファイルの読み込み
    ret = pd.read_excel(path_file, sn_list[i], header=0, index_col=None,skiprows=[1])#headerを修正
    ret = ret.rename(columns={'企画ペース\n完成版': '企画ペース',
                             '文字数\nＳ枠数': '文字数',
                             'S枠（数字）': '割当ごとのS枠数',
                             '割当単位':'割当'})#冬だけ
    ret["年度"]= sn_list[i][0:2]#sn_listのiつ目の要素から3番目の文字
    ret["採点回"]= sn_list[i][2]#sn_listのiつ目の要素から3番目の文字
    ret["学年"]= sn_list[i][3]#sn_listのiつ目の要素から4番目の文字    
    # Excel sheetsを空のリストの各要素に追加
    df.append(ret)
    
for i in (2,3,4): #21秋
    # Excelファイルの読み込み
    ret = pd.read_excel(path_file, sn_list[i], header=4,skiprows=[5], index_col=None)#headerを修正　,skiprows=[0,1,2,3,5]
    ret["年度"]= sn_list[i][0:2]#sn_listのiつ目の要素から3番目の文字
    ret["採点回"]= sn_list[i][2]#sn_listのiつ目の要素から3番目の文字
    ret["学年"]= sn_list[i][3]#sn_listのiつ目の要素から4番目の文字    
    # Excel sheetsを空のリストの各要素に追加
    df.append(ret)    

for i in (5,6): #21夏1〜2
    # Excelファイルの読み込み
    ret = pd.read_excel(path_file, sn_list[i],header = 1, index_col=None)#headerを修正
    ret["年度"]= sn_list[i][0:2]#sn_listのiつ目の要素から3番目の文字
    ret["採点回"]= sn_list[i][2]#sn_listのiつ目の要素から3番目の文字
    ret["学年"]= sn_list[i][3]#sn_listのiつ目の要素から4番目の文字    
    # Excel sheetsを空のリストの各要素に追加
    df.append(ret)
    
for i in range(7,len(sn_list)): #21夏3〜
    # Excelファイルの読み込み
    ret = pd.read_excel(path_file, sn_list[i],header = 1, index_col=None)#headerを修正
    ret["年度"]= sn_list[i][0:2]#sn_listのiつ目の要素から3番目の文字
    ret["採点回"]= sn_list[i][2]#sn_listのiつ目の要素から3番目の文字
    ret["学年"]= sn_list[i][3]#sn_listのiつ目の要素から4番目の文字    
    # Excel sheetsを空のリストの各要素に追加
    df.append(ret)

"""集計したデータを縦に結合する"""

dfs = df[0] #集計結果を一つのDFに結合
for i in df[1:]:
    dfs = dfs.append(i)

# rename    
dfs = dfs.rename(columns={'科目分類名': '科目',
                         '割当':'割当1'})

#データ型の変更
dfs['割当1'] = dfs.loc[:, "割当1"].astype('str')#下のfor文でKを除くために文字列型の必要あり
dfs['学年'] = dfs.loc[:, "学年"].astype('int')
dfs['年度'] = dfs.loc[:, "年度"].astype('int')


#置換
map_dictionary ={"国語" : 10, "数学" :20, "英語" : 30} 
dfs['科目コード'] = dfs['科目'].map(map_dictionary) 

dfs = dfs.replace({'最終ペース': {'ー':np.nan}})
dfs['最終ペース'] = dfs.loc[:, "最終ペース"].astype('float64')

#空白行の削除
dfs= dfs[~dfs["科目コード"].isna()]

#割当から客観とnanを削除
df = []
for i in dfs['割当1']:
    if 'K' in i:
        i = np.nan
    df.append(i)
dfs['割当1'] = df
dfs =dfs[dfs['割当1'] != "nan"]
dfs= dfs[~dfs["割当1"].isna()]

df_1 = dfs

In [119]:
#分野マスターは分野の表記揺れを統一したもの。 CRLEA様から修正版が送られてきたら、csvにして再度使用
"""
data1から分野を抜き出したcsvファイルを読み込む
2つのテーブルを結合させる
df_1:過去データを読み込んで一つのデータフレームにしたもの
bunya:分野の表記揺れをなおした列（分野名（修正））がある
"""

name_csv = "bunyamaster_v3.1.csv"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/rawdata"#Excelが置いてあるフォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_csv)#ファイルパスとファイル名

# csvファイルの読み込みと空のリストに追加
bunya = pd.read_csv(filepath_or_buffer = path_file, sep=",")
bunya = bunya.rename(columns={'分野名':'分野'})
bunya = bunya.fillna({'解答言語': '日本語'})
# merge
df_1b = pd.merge(df_1,bunya, on = ["分野","科目コード"],how = 'left')

In [120]:
df = df_1b

"""文字数　あとで数学の文字数を編集する必要あり。"""

def make_sum(input_str):
    if input_str =="":
        return np.nan
    else:
        input_list = input_str.split('＋')#+が半角か全角か異なる場合もあり。
        input_list_num = [int(s) for s in input_list] #リスト内包表記　復習
        return sum(input_list_num)


#一行に対するルールを定義する 上から順に作成して、順次テストする
x = ['文字程度','文字（説明）','字程度（日本語）','文字','※解答枠６。まとめて10点', '単語？','太枠', 'ポ採\u30001枠', 'ポ採\u30003枠','\u3000','小枠2つ','2枠', '1枠', '3枠']
y = ["語程度","語"]
z = ['5語程度×2枠', '50字以上60字以内（日本語）','30字以上40字以内']
def make_mojisu(df):
    
    #行を含む場合は行数の文字列を代入
    if '行' in str(df['文字数']):
        return df['行の文字数']
    
    #文字数に数値が入っている場合、そのまま代入
    elif str(df['文字数']).isdigit() == True: #簡単な例から場合分けする。 文字列か、数値か判定
        return int(df['文字数'])

    #ルール分けが難しい場合は直接数字で返す
    if '5語程度×2枠' in str(df['文字数']):#ここも語数だから直す
        return int(df['文字数'].replace('5語程度×2枠','10'))
    if '50字以上60字以内（日本語）' in str(df['文字数']):
        return int(df['文字数'].replace('50字以上60字以内（日本語）','50'))
    if '30字以上40字以内' in str(df['文字数']):
        return int(df['文字数'].replace('30字以上40字以内','30'))
    if '30～45語' in str(df['文字数']):#ここも語数だから直す
        return int(df['文字数'].replace('30～45語','30'))
    if '5～10語' in str(df['文字数']):#ここも語数だから直す
        return int(df['文字数'].replace('5～10語','5'))

    #数値に文字列を含む場合は文字列を除外し数値だけ返す。また意味不明は空白で返す
    for i in x:
        if i in str(df['文字数']):
            return make_sum(df['文字数'].replace(i,""))
    #「語」表記は5倍して返す。ただし、そのうち分野区分マスタに英文か和文かを示すダミー変数を作成して試したいので、ここは後で直す予定。
    for i in y:
        if i in str(df['文字数']):
            return int(df['文字数'].replace(i,""))
    else:
        return np.nan


#定義したルールをapplyでdfの全てに対して一行ずつ適応している。applyはよく使う。
df['置換後の文字数']=df.apply(make_mojisu,axis = 1)

"""解答言語が英語の場合、文字数を5倍にする"""
#5倍するラムダ関数
def times_five(df):
    if df["解答言語"]== "英語":
        return df["置換後の文字数"]*5
    else:
        return df["置換後の文字数"]
#定義したルールをapplyでdfの全てに対して一行ずつ適応している。applyはよく使う。
df['置換後の文字数5']=df.apply(times_five,axis = 1)

"""ポイント数の修正"""

name_col = "ポイント数"
lst = []
df[name_col] = df[name_col].replace('\u3000',np.nan)
df[name_col] = df[name_col].replace('',np.nan)
df[name_col] = df[name_col].replace('2+7+8(ポ5)',"2+7+8")
df[name_col] = df[name_col].replace('2+4+6（ポ3）',"2+4+6")
                             
for i in df[name_col]:
    if type(i) == str:
        x = i.split('+')
        y = list(map(int, x))
        i = sum(y)
    lst.append(i)

df['置換後のポイント数'] = lst

"""配点の修正"""
# df = raw_df
name_col = "配点"
lst = []
df[name_col] = df[name_col].replace('\u3000',np.nan)
df[name_col] = df[name_col].replace('',np.nan)
df[name_col] = df[name_col].replace('5(2+2+1)+5',10)
df[name_col] = df[name_col].replace('2*2',4)
                             
for i in df[name_col]:
    if type(i) == str:
        if '+' in i:
            x = i.split('+')
        elif '＋' in i:
            x = i.split('＋')
        y = list(map(int, x))
        i = sum(y)
    lst.append(i)

df['置換後の配点'] = lst

df_1b_cleaned = df #読み込んだデータに分野を足してデータクレンジングしたもの

"""
欠損値を分野別、科目別平均値で補完
1.　pandasでDFの形でデータを読み込む
2.　df.queryで分野2を条件に、カテゴリごとのDFに分ける
3.　分けたDFで、平均値で欠損値を埋める
4.　df.appendで分けたDFを結合
"""
df = df_1b_cleaned.copy()
list_bunya = df['分野名_修正v2'].unique()
list_fillna = ['置換後の文字数','置換後の文字数5','置換後のポイント数','置換後の配点']

df_ret=[]
for i in list_bunya:
    ret = df[df['分野名_修正v2'] == i]
    for j in list_fillna:
        ret[j] =ret[j].fillna(ret[j].mean())
    df_ret.append(ret)

dfs = df_ret[0] #集計結果を一つのDFに結合
for i in df_ret[1:]:
    dfs = dfs.append(i)

df_1b_cleaned_filled = dfs

/var/folders/ll/_gcfyf2137x3x6dwgb9nkj740000gp/T/ipykernel_7660/139618028.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ret[j] =ret[j].fillna(ret[j].mean())


In [121]:
"""
Excelの書き出し 
"""

df = df_1b_cleaned

name_excel_output = "crlea_bunya_dm_0613received_0613cleaned_test.csv"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス

df.to_csv('{}/{}'.format(path_folder,name_excel_output),encoding='utf-8-sig',index=False)

name_excel_output = "crlea_bunya_dm_0613received_0613cleaned_test.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#Excelが置いてあるフォルダパス


with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df.to_excel(writer, sheet_name='data1',encoding='utf-8-sig', index = False)
    
df = df_1b_cleaned_filled

name_excel_output = "crlea_bunya_dm_0613received_0613cleaned_filled_test.csv"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス

df.to_csv('{}/{}'.format(path_folder,name_excel_output),encoding='utf-8-sig',index=False)

name_excel_output = "crlea_bunya_dm_0613received_0613cleaned_filled_test.xlsx"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#Excelが置いてあるフォルダパス


with pd.ExcelWriter('{}/{}'.format(path_folder,name_excel_output)) as writer:
    df.to_excel(writer, sheet_name='data1',encoding='utf-8-sig', index = False)

## 機械学習

#データ読み込み マークダウンにしてあるから必要ならコードセルにする
"""
data1を加工したcsvファイルを読み込む場合　
"""

name_csv = "crlea_bunya_dm_0613received_0613cleaned_test.csv" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#データが置いてあるフォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_csv)#ファイルパスとファイル名

# csvファイルの読み込みと空のリストに追加
df = pd.read_csv(filepath_or_buffer = path_file, sep=",",
                 usecols=['科目',
                          '分野名_修正v1',
                          '分野名_修正v2',
                          'ポイント採点',
                          '年度',
                          '採点回',
                          '学年',
                          '置換後のポイント数',
                          '置換後の文字数',
                          '置換後の文字数5',
                          '解答言語',
                          '置換後の配点',
                          '企画ペース',
                          '最終ペース'])
#列のリネーム
df = df.rename(columns={'分野名_修正v1':'分野1',
                        '分野名_修正v2':'分野2',
                        '置換後のポイント数':'ポイント数',
                        '置換後の文字数':'文字数',#英語1単語も1文字として数えた
                        '置換後の文字数5':'文字数5',#英語１単語を5文字とした。前の分析ではこっち。精度を比較する。
                        '置換後の配点':'配点'})

In [130]:
"""データ選択"""

def select_data(df):
    df = df[['科目',
        '分野名_修正v1',
        '分野名_修正v2',
        'ポイント採点',
        '年度',
        '採点回',
        '学年',
        '置換後のポイント数',
        '置換後の文字数',
        '置換後の文字数5',
         '解答言語',
        '置換後の配点',
        '企画ペース',
        '最終ペース']]
    df = df.rename(columns={'分野名_修正v1':'分野1',
                        '分野名_修正v2':'分野2',
                        '置換後のポイント数':'ポイント数',
                        '置換後の文字数':'文字数',#英語1単語も1文字として数えた
                        '置換後の文字数5':'文字数5',#英語１単語を5文字とした。前の分析ではこっち。精度を比較する。
                        '置換後の配点':'配点'})
    return df

df_ml = select_data(df_1b_cleaned)
df_ml_filled = select_data(df_1b_cleaned_filled)

### DecisionTreeRegressorメソッドの概要

|引数名|概要|デフォルト|
| :---- | :---- | :---- |
|criterion|不純度を測定する基準（平均二乗誤差、平均絶対誤差など）|‘mse’|
|splitter|条件探索アルゴリズムを選択するオプション（’best’と’rondom’が指定可能）|‘best’|
|max_depth|決定木のノード深さの制限値。ツリーが深くなりすぎて過学習の状態に陥った際は、このパラメータが正則化の役割を果たす。|None|
|min_samples_split|ノードを分割するために必要なサンプルの最小値|2|
|min_samples_leaf|1ノードの深さを作成するために必要となるデータ数の最小値。指定した値以上のデータ数を持たないノードは作られない。|1|
|min_weight_fraction_leaf|サンプルの重みを考慮した上でのmin_samples_leafに該当|0.0|
|max_features|ランダムに指定する説明変数の数(全ての説明変数がモデル学習に活用されるわけではなく、ランダムに割り振られる）|None|
|random_state|乱数シード|None|
|max_leaf_nodes|作成される決定木の葉の数を、指定した値以下に制御する|None|
|min_impurity_decrease|決定木の成長の早期停止するための閾値。不純度が指定の値より減少した場合、ノードを分岐し、不純度が指定の値より減少しなければ分岐を抑制。|0.0|
|ccp_alpha|ccp_alphaが大きいほどプルーニングされるノードの数が増加。プルーニングとは、精度低下をできるだけ抑えながら過剰な重みを排除するプロセスを指す。|0.0|


### 関数の定義

In [100]:
"""
1単語を文字数5で数える
機械学習用データを準備する関数
df[0]:21夏1年生モデル：検証データ ＝ 21年夏1学年
df[1]:21夏2年生モデル：検証データ ＝ 21年夏2学年
df[2]:21夏3年生モデル：検証データ ＝ 21年夏3学年
df[3]:21夏学年混合モデル：検証データ ＝ 21年夏全学年
"""

def make_mldata(df,str):
    """
    df:使用するデータ
    str:抽出する科目
    1.選択した科目を抽出
    2.one hot encoding
    3.説明変数、目的変数でデータフレームを分離
    4.訓練用、検証用にデータフレームを分離
        1.21夏1年生モデル：検証データ ＝ 21年夏1学年
        2.21夏2年生モデル：検証データ ＝ 21年夏2学年
        3.21夏3年生モデル：検証データ ＝ 21年夏3学年
        4.21夏学年混合モデル：検証データ ＝ 21年夏全学年
    5.説明変数から採点回を削除 
    """
    #0.NAN remove 追加した
    df = df.dropna(subset=['ポイント数', '配点', '文字数'])
    df.loc[:,'ポイント数'] =df.loc[:,'ポイント数'].round(0).astype(int)
    df.loc[:,'配点'] = df.loc[:,'配点'].round(0).astype(int)
    df.loc[:,'文字数'] = df.loc[:,'文字数'].round(0).astype(int)
    df.loc[:,'文字数5'] = df.loc[:,'文字数5'].round(0).astype(int)
    #1.科目を抽出し、科目と分野1の列を削除
    df = df[df['科目']== str].drop(columns = ["科目","分野1","文字数","解答言語"])#解答言語を考慮しない
    
    #2.名義変数のエンコーディング pandas get_dummies関数でone hot encording
    df = pd.get_dummies(df)
    
    #3.上記のデータセットを説明変数と目的変数で分ける
    df_X = df.drop(columns = "最終ペース")#目的変数を除外した（説明変数だけ含む）データフレーム
    df_y = df.loc[:,['年度','採点回_夏','学年','最終ペース']]#目的変数と選択用変数だけ含むデータフレーム
    
    #4.データフレームの分離 不恰好なのをどうにかしたい
    """秋冬が訓練用"""
    #訓練用説明変数
    X_train_1 = df_X[~((df_X['採点回_夏'] == 1)&(df_X['年度']==21)&(df_X['学年']==1))]
    X_train_2 = df_X[~((df_X['採点回_夏'] == 1)&(df_X['年度']==21)&(df_X['学年']==2))]
    X_train_3 = df_X[~((df_X['採点回_夏'] == 1)&(df_X['年度']==21)&(df_X['学年']==3))]
    X_train_4 = df_X[~((df_X['採点回_夏'] == 1)&(df_X['年度']==21))]
    #訓練用目的変数
    y_train_1 = df_y[~((df_y['採点回_夏'] == 1)&(df_y['年度']==21)&(df_y['学年']==1))]
    y_train_2 = df_y[~((df_y['採点回_夏'] == 1)&(df_y['年度']==21)&(df_y['学年']==2))]
    y_train_3 = df_y[~((df_y['採点回_夏'] == 1)&(df_y['年度']==21)&(df_y['学年']==3))]
    y_train_4 = df_y[~((df_y['採点回_夏'] == 1)&(df_y['年度']==21))]

    """夏が検証用"""
    #訓練用説明変数
    X_test_1 = df_X[(df_X['採点回_夏'] == 1)&(df_X['年度']==21)&(df_X['学年']==1)]
    X_test_2 = df_X[(df_X['採点回_夏'] == 1)&(df_X['年度']==21)&(df_X['学年']==2)]
    X_test_3 = df_X[(df_X['採点回_夏'] == 1)&(df_X['年度']==21)&(df_X['学年']==3)]
    X_test_4 = df_X[(df_X['採点回_夏'] == 1)&(df_X['年度']==21)]
    #訓練用目的変数
    y_test_1 = df_y[(df_y['採点回_夏'] == 1)&(df_y['年度']==21)&(df_y['学年']==1)]
    y_test_2 = df_y[(df_y['採点回_夏'] == 1)&(df_y['年度']==21)&(df_y['学年']==2)]
    y_test_3 = df_y[(df_y['採点回_夏'] == 1)&(df_y['年度']==21)&(df_y['学年']==3)]
    y_test_4 = df_y[(df_y['採点回_夏'] == 1)&(df_y['年度']==21)]
   
    #5.不要な変数を削除
    rem_cols_x = ["企画ペース","採点回_夏","採点回_秋","採点回_冬","年度"]
    rem_cols_xb = ["採点回_夏","採点回_秋","採点回_冬","年度"]
    rem_cols_y = ["採点回_夏","年度","学年"]

    #説明変数
    X_train_1 = X_train_1.drop(columns = rem_cols_x)
    X_train_2 = X_train_2.drop(columns = rem_cols_x)
    X_train_3 = X_train_3.drop(columns = rem_cols_x)
    X_train_4 = X_train_4.drop(columns = rem_cols_x)

    #出力用に企画ペースを残している
    X_test_1b = X_test_1.drop(columns = rem_cols_xb)
    X_test_2b = X_test_2.drop(columns = rem_cols_xb)
    X_test_3b = X_test_3.drop(columns = rem_cols_xb)
    X_test_4b = X_test_4.drop(columns = rem_cols_xb)    
    
    X_test_1 = X_test_1.drop(columns = rem_cols_x)
    X_test_2 = X_test_2.drop(columns = rem_cols_x)
    X_test_3 = X_test_3.drop(columns = rem_cols_x)
    X_test_4 = X_test_4.drop(columns = rem_cols_x)
         
    #目的変数
    y_train_1 = y_train_1.drop(columns = rem_cols_y)
    y_train_2 = y_train_2.drop(columns = rem_cols_y)
    y_train_3 = y_train_3.drop(columns = rem_cols_y)
    y_train_4 = y_train_4.drop(columns = rem_cols_y)

    y_test_1 = y_test_1.drop(columns = rem_cols_y)
    y_test_2 = y_test_2.drop(columns = rem_cols_y)
    y_test_3 = y_test_3.drop(columns = rem_cols_y)
    y_test_4 = y_test_4.drop(columns = rem_cols_y)
    
    #各モデル用データを各データフレームにまとめ、それらをさらにデータフレームにまとめて返す。
    df_res=[]
    data1=[X_train_1,y_train_1,X_test_1,y_test_1,X_test_1b]#1年生モデルデータ なぜかdata1はリストとして入っていて、X_trainとかはdf。
    data2=[X_train_2,y_train_2,X_test_2,y_test_2,X_test_2b]#2年生モデルデータ
    data3=[X_train_3,y_train_3,X_test_3,y_test_3,X_test_3b]#3年生モデルデータ
    data4=[X_train_4,y_train_4,X_test_4,y_test_4,X_test_4b]#学年混合モデルデータ
    df_res = [data1, data2, data3, data4]

    return df_res

In [146]:
"""
1単語を文字数5で数える。数学の分野を大まかに区分。
機械学習用データを準備する関数
df[0]:21夏1年生モデル：検証データ ＝ 21年夏1学年
df[1]:21夏2年生モデル：検証データ ＝ 21年夏2学年
df[2]:21夏3年生モデル：検証データ ＝ 21年夏3学年
df[3]:21夏学年混合モデル：検証データ ＝ 21年夏全学年
"""

def make_mldata_all(df,str):
    """
    df:使用するデータ
    str:抽出する科目
    1.選択した科目を抽出
    2.one hot encoding
    3.説明変数、目的変数でデータフレームを分離
    4.訓練用、検証用にデータフレームを分離
        1.21夏1年生モデル：検証データ ＝ 21年夏1学年
        2.21夏2年生モデル：検証データ ＝ 21年夏2学年
        3.21夏3年生モデル：検証データ ＝ 21年夏3学年
        4.21夏学年混合モデル：検証データ ＝ 21年夏全学年
    5.説明変数から採点回を削除 
    """
    #0.NAN remove 追加した
    df = df.dropna(subset=['ポイント数', '配点', '文字数'])
    df.loc[:,'ポイント数'] =df.loc[:,'ポイント数'].round(0).astype(int)
    df.loc[:,'配点'] = df.loc[:,'配点'].round(0).astype(int)
    df.loc[:,'文字数'] = df.loc[:,'文字数'].round(0).astype(int)
    df.loc[:,'文字数5'] = df.loc[:,'文字数5'].round(0).astype(int)
    #1.科目を抽出し、科目と分野1の列を削除
    df = df.drop(columns = ["分野1","文字数","解答言語"])#解答言語を考慮しない
    
    #2.名義変数のエンコーディング pandas get_dummies関数でone hot encording
    df = pd.get_dummies(df)
    
    #3.上記のデータセットを説明変数と目的変数で分ける
    df_X = df.drop(columns = "最終ペース")#目的変数を除外した（説明変数だけ含む）データフレーム
    df_y = df.loc[:,['年度','採点回_夏','学年','最終ペース']]#目的変数と選択用変数だけ含むデータフレーム
    
    #4.データフレームの分離 不恰好なのをどうにかしたい
    """秋冬が訓練用"""
    #訓練用説明変数
    X_train_1 = df_X[~((df_X['採点回_夏'] == 1)&(df_X['年度']==21)&(df_X['学年']==1))]
    X_train_2 = df_X[~((df_X['採点回_夏'] == 1)&(df_X['年度']==21)&(df_X['学年']==2))]
    X_train_3 = df_X[~((df_X['採点回_夏'] == 1)&(df_X['年度']==21)&(df_X['学年']==3))]
    X_train_4 = df_X[~((df_X['採点回_夏'] == 1)&(df_X['年度']==21))]
    #訓練用目的変数
    y_train_1 = df_y[~((df_y['採点回_夏'] == 1)&(df_y['年度']==21)&(df_y['学年']==1))]
    y_train_2 = df_y[~((df_y['採点回_夏'] == 1)&(df_y['年度']==21)&(df_y['学年']==2))]
    y_train_3 = df_y[~((df_y['採点回_夏'] == 1)&(df_y['年度']==21)&(df_y['学年']==3))]
    y_train_4 = df_y[~((df_y['採点回_夏'] == 1)&(df_y['年度']==21))]

    """夏が検証用"""
    #訓練用説明変数
    X_test_1 = df_X[(df_X['採点回_夏'] == 1)&(df_X['年度']==21)&(df_X['学年']==1)]
    X_test_2 = df_X[(df_X['採点回_夏'] == 1)&(df_X['年度']==21)&(df_X['学年']==2)]
    X_test_3 = df_X[(df_X['採点回_夏'] == 1)&(df_X['年度']==21)&(df_X['学年']==3)]
    X_test_4 = df_X[(df_X['採点回_夏'] == 1)&(df_X['年度']==21)]
    #訓練用目的変数
    y_test_1 = df_y[(df_y['採点回_夏'] == 1)&(df_y['年度']==21)&(df_y['学年']==1)]
    y_test_2 = df_y[(df_y['採点回_夏'] == 1)&(df_y['年度']==21)&(df_y['学年']==2)]
    y_test_3 = df_y[(df_y['採点回_夏'] == 1)&(df_y['年度']==21)&(df_y['学年']==3)]
    y_test_4 = df_y[(df_y['採点回_夏'] == 1)&(df_y['年度']==21)]
   
    #5.不要な変数を削除
    rem_cols_x = ["企画ペース","採点回_夏","採点回_秋","採点回_冬","年度"]
    rem_cols_xb = ["採点回_夏","採点回_秋","採点回_冬","年度"]
    rem_cols_y = ["採点回_夏","年度","学年"]

    #説明変数
    X_train_1 = X_train_1.drop(columns = rem_cols_x)
    X_train_2 = X_train_2.drop(columns = rem_cols_x)
    X_train_3 = X_train_3.drop(columns = rem_cols_x)
    X_train_4 = X_train_4.drop(columns = rem_cols_x)

    #出力用に企画ペースを残している
    X_test_1b = X_test_1.drop(columns = rem_cols_xb)
    X_test_2b = X_test_2.drop(columns = rem_cols_xb)
    X_test_3b = X_test_3.drop(columns = rem_cols_xb)
    X_test_4b = X_test_4.drop(columns = rem_cols_xb)    
    
    X_test_1 = X_test_1.drop(columns = rem_cols_x)
    X_test_2 = X_test_2.drop(columns = rem_cols_x)
    X_test_3 = X_test_3.drop(columns = rem_cols_x)
    X_test_4 = X_test_4.drop(columns = rem_cols_x)
         
    #目的変数
    y_train_1 = y_train_1.drop(columns = rem_cols_y)
    y_train_2 = y_train_2.drop(columns = rem_cols_y)
    y_train_3 = y_train_3.drop(columns = rem_cols_y)
    y_train_4 = y_train_4.drop(columns = rem_cols_y)

    y_test_1 = y_test_1.drop(columns = rem_cols_y)
    y_test_2 = y_test_2.drop(columns = rem_cols_y)
    y_test_3 = y_test_3.drop(columns = rem_cols_y)
    y_test_4 = y_test_4.drop(columns = rem_cols_y)
    
    #各モデル用データを各データフレームにまとめ、それらをさらにデータフレームにまとめて返す。
    df_res=[]
    data1=[X_train_1,y_train_1,X_test_1,y_test_1,X_test_1b]#1年生モデルデータ なぜかdata1はリストとして入っていて、X_trainとかはdf。
    data2=[X_train_2,y_train_2,X_test_2,y_test_2,X_test_2b]#2年生モデルデータ
    data3=[X_train_3,y_train_3,X_test_3,y_test_3,X_test_3b]#3年生モデルデータ
    data4=[X_train_4,y_train_4,X_test_4,y_test_4,X_test_4b]#学年混合モデルデータ
    df_res = [data1, data2, data3, data4]

    return df_res

In [144]:
"""
解答言語と文字数,分野2
機械学習用データを準備する関数
df[0]:21夏1年生モデル：検証データ ＝ 21年夏1学年
df[1]:21夏2年生モデル：検証データ ＝ 21年夏2学年
df[2]:21夏3年生モデル：検証データ ＝ 21年夏3学年
df[3]:21夏学年混合モデル：検証データ ＝ 21年夏全学年
"""

def make_mldata2(df,str):
    """
    df:使用するデータ
    str:抽出する科目
    1.選択した科目を抽出
    2.one hot encoding
    3.説明変数、目的変数でデータフレームを分離
    4.訓練用、検証用にデータフレームを分離
        1.21夏1年生モデル：検証データ ＝ 21年夏1学年
        2.21夏2年生モデル：検証データ ＝ 21年夏2学年
        3.21夏3年生モデル：検証データ ＝ 21年夏3学年
        4.21夏学年混合モデル：検証データ ＝ 21年夏全学年
    5.説明変数から採点回を削除 
    """
    #0.NAN remove 追加した
    df = df.dropna(subset=['ポイント数', '配点', '文字数'])
    df.loc[:,'ポイント数'] =df.loc[:,'ポイント数'].round(0).astype(int)
    df.loc[:,'配点'] = df.loc[:,'配点'].round(0).astype(int)
    df.loc[:,'文字数'] = df.loc[:,'文字数'].round(0).astype(int)
    df.loc[:,'文字数5'] = df.loc[:,'文字数5'].round(0).astype(int)
    #1.科目を抽出し、科目と分野1の列を削除
    df = df[df['科目']== str].drop(columns = ["科目","分野1","文字数5"])#解答言語を考慮しない
    
    #2.名義変数のエンコーディング pandas get_dummies関数でone hot encording
    df = pd.get_dummies(df)
    
    #3.上記のデータセットを説明変数と目的変数で分ける
    df_X = df.drop(columns = "最終ペース")#目的変数を除外した（説明変数だけ含む）データフレーム
    df_y = df.loc[:,['年度','採点回_夏','学年','最終ペース']]#目的変数と選択用変数だけ含むデータフレーム
    
    #4.データフレームの分離 不恰好なのをどうにかしたい
    """秋冬が訓練用"""
    #訓練用説明変数
    X_train_1 = df_X[~((df_X['採点回_夏'] == 1)&(df_X['年度']==21)&(df_X['学年']==1))]
    X_train_2 = df_X[~((df_X['採点回_夏'] == 1)&(df_X['年度']==21)&(df_X['学年']==2))]
    X_train_3 = df_X[~((df_X['採点回_夏'] == 1)&(df_X['年度']==21)&(df_X['学年']==3))]
    X_train_4 = df_X[~((df_X['採点回_夏'] == 1)&(df_X['年度']==21))]
    #訓練用目的変数
    y_train_1 = df_y[~((df_y['採点回_夏'] == 1)&(df_y['年度']==21)&(df_y['学年']==1))]
    y_train_2 = df_y[~((df_y['採点回_夏'] == 1)&(df_y['年度']==21)&(df_y['学年']==2))]
    y_train_3 = df_y[~((df_y['採点回_夏'] == 1)&(df_y['年度']==21)&(df_y['学年']==3))]
    y_train_4 = df_y[~((df_y['採点回_夏'] == 1)&(df_y['年度']==21))]

    """夏が検証用"""
    #訓練用説明変数
    X_test_1 = df_X[(df_X['採点回_夏'] == 1)&(df_X['年度']==21)&(df_X['学年']==1)]
    X_test_2 = df_X[(df_X['採点回_夏'] == 1)&(df_X['年度']==21)&(df_X['学年']==2)]
    X_test_3 = df_X[(df_X['採点回_夏'] == 1)&(df_X['年度']==21)&(df_X['学年']==3)]
    X_test_4 = df_X[(df_X['採点回_夏'] == 1)&(df_X['年度']==21)]
    #訓練用目的変数
    y_test_1 = df_y[(df_y['採点回_夏'] == 1)&(df_y['年度']==21)&(df_y['学年']==1)]
    y_test_2 = df_y[(df_y['採点回_夏'] == 1)&(df_y['年度']==21)&(df_y['学年']==2)]
    y_test_3 = df_y[(df_y['採点回_夏'] == 1)&(df_y['年度']==21)&(df_y['学年']==3)]
    y_test_4 = df_y[(df_y['採点回_夏'] == 1)&(df_y['年度']==21)]
   
    #5.不要な変数を削除
    rem_cols_x = ["企画ペース","採点回_夏","採点回_秋","採点回_冬","年度"]
    rem_cols_xb = ["採点回_夏","採点回_秋","採点回_冬","年度"]
    rem_cols_y = ["採点回_夏","年度","学年"]

    #説明変数
    X_train_1 = X_train_1.drop(columns = rem_cols_x)
    X_train_2 = X_train_2.drop(columns = rem_cols_x)
    X_train_3 = X_train_3.drop(columns = rem_cols_x)
    X_train_4 = X_train_4.drop(columns = rem_cols_x)

    #出力用に企画ペースを残している
    X_test_1b = X_test_1.drop(columns = rem_cols_xb)
    X_test_2b = X_test_2.drop(columns = rem_cols_xb)
    X_test_3b = X_test_3.drop(columns = rem_cols_xb)
    X_test_4b = X_test_4.drop(columns = rem_cols_xb)    
    
    X_test_1 = X_test_1.drop(columns = rem_cols_x)
    X_test_2 = X_test_2.drop(columns = rem_cols_x)
    X_test_3 = X_test_3.drop(columns = rem_cols_x)
    X_test_4 = X_test_4.drop(columns = rem_cols_x)
         
    #目的変数
    y_train_1 = y_train_1.drop(columns = rem_cols_y)
    y_train_2 = y_train_2.drop(columns = rem_cols_y)
    y_train_3 = y_train_3.drop(columns = rem_cols_y)
    y_train_4 = y_train_4.drop(columns = rem_cols_y)

    y_test_1 = y_test_1.drop(columns = rem_cols_y)
    y_test_2 = y_test_2.drop(columns = rem_cols_y)
    y_test_3 = y_test_3.drop(columns = rem_cols_y)
    y_test_4 = y_test_4.drop(columns = rem_cols_y)
    
    #各モデル用データを各データフレームにまとめ、それらをさらにデータフレームにまとめて返す。
    df_res=[]
    data1=[X_train_1,y_train_1,X_test_1,y_test_1,X_test_1b]#1年生モデルデータ なぜかdata1はリストとして入っていて、X_trainとかはdf。
    data2=[X_train_2,y_train_2,X_test_2,y_test_2,X_test_2b]#2年生モデルデータ
    data3=[X_train_3,y_train_3,X_test_3,y_test_3,X_test_3b]#3年生モデルデータ
    data4=[X_train_4,y_train_4,X_test_4,y_test_4,X_test_4b]#学年混合モデルデータ
    df_res = [data1, data2, data3, data4]

    return df_res

In [64]:
"""
https://stackoverflow.com/questions/50607740/reverse-a-get-dummies-encoding-in-pandas
ダミー変数を元に戻す。エクセル出力用
"""

def undummify(df, prefix_sep="_"):
    cols2collapse = {
        item.split(prefix_sep)[0]: (prefix_sep in item) for item in df.columns
    }
    series_list = []
    for col, needs_to_collapse in cols2collapse.items():
        if needs_to_collapse:
            undummified = (
                df.filter(like=col)
                .idxmax(axis=1)
                .apply(lambda x: x.split(prefix_sep, maxsplit=1)[1])
                .rename(col)
            )
            series_list.append(undummified)
        else:
            series_list.append(df[col])
    undummified_df = pd.concat(series_list, axis=1)
    return undummified_df

In [68]:
"""
make_mldataのseriesを使って、予測値を出し、予測値、最終ペース、差分、乖離度（予測値/最終ペース）を列に追加したdfを返す。
"""
def test_model2(df,a=2,b=1,c=None):
    X_train= df[0]
    y_train= df[1]
    X_test = df[2]
    y_test = df[3]
    X_test_b = df[4]
    model = DecisionTreeRegressor(criterion='mse', 
                                   splitter='best', 
                                   max_depth=c, 
                                   min_samples_split=a, #3,4,5とか？
                                   min_samples_leaf=b,#2とか 
                                   min_weight_fraction_leaf=0.0,
                                   max_features=None, 
                                   random_state=None, 
                                   max_leaf_nodes=None, 
                                   min_impurity_decrease=0.0, 
                                   ccp_alpha=0.0
                                  )

    #上記のパラメータでモデルを学習する
    model.fit(X_train, y_train)
    y_pred  = model.predict(X_test)
    #得た結果の樹形図を表示する
    
    #得た結果を学習データとマージしてデータフレームで返す
    df_res=[]
    
    #検証に使った説明変数データのダミー変数を元に戻す
    df_res = undummify(X_test_b)  #企画ペースを入れるためにここをいじった。元はX_test
    df_res.loc[:,'AI想定ペース']= y_pred #上のデータに予測値をマージ
    df_res.loc[:,'最終ペース']= y_test
    df_res.loc[:,'誤差']= df_res['AI想定ペース']-df_res['最終ペース']#実測値と予測値の差分を列に追加 
    df_res.loc[:,'AI乖離度']= df_res['AI想定ペース']/df_res['最終ペース']#実測値と予測値の差分を列に追加 
    df_res.loc[:,'元の乖離度']= df_res['企画ペース']/df_res['最終ペース']#実測値と予測値の差分を列に追加 
            
    # 最終ペースをランク分け
    x = "最終ペース"
    conditions = [
        (df_res[x] >= 600),
        (df_res[x] >= 500),
        (df_res[x] >= 400),
        (df_res[x] >= 300),
        (df_res[x] >= 200),
        (df_res[x] >= 100),
        (df_res[x] >= 0)
         ]

    choices = ["600~", "500~600", "400~500", "300~400", "200~300", "100~200","0~100"]

    df_res.loc[:,'最終ランク'] = np.select(conditions, choices, default = 0)
    return df_res

In [65]:
#使用ライブラリ
# from sklearn import tree
# import graphviz
"""
make_mldataのデータを使って樹形図を作成し、pdfに保存する。

df_input:入力データ
a:作成するファイルの3つめの単語
b:作成するファイルの4つめの単語
1. 科目のリストを作成
2. モデルのリストを作成
3. 科目、モデルの順で樹形図を作成し、保存
"""
def make_leaf3(df_input,a=2,b=1,c=None):
    kamoku_name=["国語","英語","数学"]
    model_name=["1年","2年","3年","学年混合"]
    model = DecisionTreeRegressor(criterion='mse', 
                                   splitter='best', 
                                   max_depth=c, 
                                   min_samples_split=a, #3,4,5とか？
                                   min_samples_leaf=b,#2とか 
                                   min_weight_fraction_leaf=0.0,
                                   max_features=None, 
                                   random_state=None, 
                                   max_leaf_nodes=None, 
                                   min_impurity_decrease=0.0, 
                                   ccp_alpha=0.0
                                  )
    df = []
    for i in kamoku_name:
        df = make_mldata(df_input,i)
        for j in range(0,len(model_name)):
            X_train= df[j][0]
            y_train= df[j][1]
            X_test = df[j][2]
            y_test = df[j][3]
            X_train_feature_names = X_train.columns.values.tolist() 
            
            #上記のパラメータでモデルを学習する
            model.fit(X_train, y_train)

            dot_data = tree.export_graphviz(model, out_file=None,
                                            feature_names= X_train_feature_names,
                                            # class_names=iris.target_names,
                                            filled=True, rounded=True, special_characters=True
                                           )
            graph = graphviz.Source(dot_data) 
            """グラフの保存"""
            name_file = "DTs_{}_{}_mss{}_msl{}_md{}".format(i,model_name[j],str(a),str(b),str(c)) #ファイル名
            path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス
            path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名
            graph.render(path_file) 

# 結果出力

In [134]:
"""欠損値を補完したデータの予測値"""
#予測値と統計値を各科目のモデルごとにエクセルシートに出力


kamoku=["国語","英語","数学"]#,"数学"は文字数がないため、dropnaで除外してしまっている。前処理で変更する必要あり。
model=["1年","2年","3年","学年混合"]

mss= 2#min sample split
msl= 1#min sample leaf
md=None #max depth
test_num="June3"

name_file = "DTs_{}_mss{}_msl{}_md{}.xlsx".format(str(test_num),str(mss),str(msl),str(md)) #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス
# path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名

# 入力データ
df = df_ml_filled
df1=[[[],[],[],[]],[[],[],[],[]],[[],[],[],[]]]
df2=[[[],[],[],[]],[[],[],[],[]],[[],[],[],[]]]
for i,n in enumerate(kamoku):
    for j,m in enumerate(model):
        df1[i][j] = test_model2(make_mldata(df,kamoku[i])[j],mss,msl,md)
        df2[i][j] = pandas_tool.summary(df1[i][j].loc[:,['最終ペース','企画ペース','AI想定ペース','誤差','AI乖離度','元の乖離度']])[0]
        
with pd.ExcelWriter('{}/{}'.format(path_folder,name_file)) as writer:
    for i,n in enumerate(kamoku):
        for j,m in enumerate(model):
            s_names=['{}_{}_values'.format(kamoku[i],model[j]),'{}_{}_stats'.format(kamoku[i],model[j])]
            df1[i][j].to_excel(writer, sheet_name='{}'.format(s_names[0]))
            df2[i][j].to_excel(writer, sheet_name='{}'.format(s_names[1]))
            
            
# =IFS(M2>=600,6,M2>=500,5,M2>=400,4,M2>=300,3,M2>=200,2,M2>=100,1,TRUE,0)

/Users/s.ogura/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/Users/s.ogura/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/Users/s.ogura/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [145]:
"""欠損値を補完したデータの予測値"""
#予測値と統計値を各科目のモデルごとにエクセルシートに出力


kamoku=["国語","英語","数学"]#,"数学"は文字数がないため、dropnaで除外してしまっている。前処理で変更する必要あり。
model=["1年","2年","3年","学年混合"]

mss= 2#min sample split
msl= 1#min sample leaf
md=None #max depth
test_num="June3_解答言語"

name_file = "DTs_{}_mss{}_msl{}_md{}.xlsx".format(str(test_num),str(mss),str(msl),str(md)) #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス
# path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名

# 入力データ
df = df_ml_filled
df1=[[[],[],[],[]],[[],[],[],[]],[[],[],[],[]]]
df2=[[[],[],[],[]],[[],[],[],[]],[[],[],[],[]]]
for i,n in enumerate(kamoku):
    for j,m in enumerate(model):
        df1[i][j] = test_model2(make_mldata2(df,kamoku[i])[j],mss,msl,md)
        df2[i][j] = pandas_tool.summary(df1[i][j].loc[:,['最終ペース','企画ペース','AI想定ペース','誤差','AI乖離度','元の乖離度']])[0]
        
with pd.ExcelWriter('{}/{}'.format(path_folder,name_file)) as writer:
    for i,n in enumerate(kamoku):
        for j,m in enumerate(model):
            s_names=['{}_{}_values'.format(kamoku[i],model[j]),'{}_{}_stats'.format(kamoku[i],model[j])]
            df1[i][j].to_excel(writer, sheet_name='{}'.format(s_names[0]))
            df2[i][j].to_excel(writer, sheet_name='{}'.format(s_names[1]))
            
            
# =IFS(M2>=600,6,M2>=500,5,M2>=400,4,M2>=300,3,M2>=200,2,M2>=100,1,TRUE,0)

/Users/s.ogura/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/Users/s.ogura/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/Users/s.ogura/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [147]:
"""欠損値を補完したデータの予測値"""
#予測値と統計値を各科目のモデルごとにエクセルシートに出力


kamoku=["国語","英語","数学"]#,"数学"は文字数がないため、dropnaで除外してしまっている。前処理で変更する必要あり。
model=["1年","2年","3年","学年混合"]

mss= 2#min sample split
msl= 1#min sample leaf
md=None #max depth
test_num="June3_全科目"

name_file = "DTs_{}_mss{}_msl{}_md{}.xlsx".format(str(test_num),str(mss),str(msl),str(md)) #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス
# path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名

# 入力データ
df = df_ml_filled
df1=[[[],[],[],[]],[[],[],[],[]],[[],[],[],[]]]
df2=[[[],[],[],[]],[[],[],[],[]],[[],[],[],[]]]
for i,n in enumerate(kamoku):
    for j,m in enumerate(model):
        df1[i][j] = test_model2(make_mldata_all(df,kamoku[i])[j],mss,msl,md)
        df2[i][j] = pandas_tool.summary(df1[i][j].loc[:,['最終ペース','企画ペース','AI想定ペース','誤差','AI乖離度','元の乖離度']])[0]
        
with pd.ExcelWriter('{}/{}'.format(path_folder,name_file)) as writer:
    for i,n in enumerate(kamoku):
        for j,m in enumerate(model):
            s_names=['{}_{}_values'.format(kamoku[i],model[j]),'{}_{}_stats'.format(kamoku[i],model[j])]
            df1[i][j].to_excel(writer, sheet_name='{}'.format(s_names[0]))
            df2[i][j].to_excel(writer, sheet_name='{}'.format(s_names[1]))
            
            
# =IFS(M2>=600,6,M2>=500,5,M2>=400,4,M2>=300,3,M2>=200,2,M2>=100,1,TRUE,0)

/Users/s.ogura/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/Users/s.ogura/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/Users/s.ogura/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a cop